<a href="https://colab.research.google.com/github/chm231/TRG25/blob/main/Greatcell_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ───────────────────────────────────────────────────────────────
# 0. MATERIAL CONSTANTS (dimensional)
# ----------------------------------------------------------------
E   = 83.9          # Young’s modulus (MPa)
nu  = 0.21          # Poisson’s ratio
lmbd = E * nu / ((1 + nu) * (1 - 2 * nu))   # Lamé λ
mu   = E * 0.5 / (1 + nu)                   # Lamé μ

# ───────────────────────────────────────────────────────────────
# 1. GEOMETRY & EXTERNAL TRACTION (32 segments)
# ----------------------------------------------------------------
R_disk = 0.098  # disk radius (m)

# 16-point eccentric edge loading values (MPa)
sigma_pee_16 = np.array(
    [-10.0, -9.95, -7.80, -3.82, -1.00, -1.17, -4.46, -6.64] * 2,
    dtype=np.float32,
)
# Interpolate to 32 segments
sigma32 = np.zeros(32, dtype=np.float32)
sigma32[0::2]   = sigma_pee_16
sigma32[1:-1:2] = 0.5 * (sigma_pee_16[:-1] + sigma_pee_16[1:])
sigma32[31]     = 0.5 * (sigma_pee_16[-1] + sigma_pee_16[0])

# Compute segment start/end angles (deg)
arc_start = np.empty(32, dtype=np.float32)
arc_end   = np.empty(32, dtype=np.float32)
s = -9.375
for i in range(32):
    arc_start[i] = s
    s += 18.75 if i % 2 == 0 else 3.75
    arc_end[i] = s

# Move to torch tensors
ARC_START = torch.tensor(arc_start, device=device)
ARC_END   = torch.tensor(arc_end,   device=device)
SIGMA32   = torch.tensor(sigma32,   device=device)

# Function: normal traction σ_n(θ)
def sigma_of_angle(X: torch.Tensor) -> torch.Tensor:
    """
    X: (N,2) array of boundary points
    returns: (N,1) normal traction at each point
    """
    ang = torch.atan2(X[:,1], X[:,0]) * 180.0 / np.pi   # rad→deg
    ang = torch.where(ang < -9.375, ang + 360.0, ang)   # wrap-around
    ang = ang.unsqueeze(1)                              # shape (N,1)
    # boolean mask per segment
    cond      = (ang >= ARC_START) & (ang < ARC_END)
    cond_last = (ang >= ARC_START[-1]) & (ang <= ARC_END[-1])
    cond      = torch.cat([cond[:,:-1], cond_last], dim=1)
    return torch.sum(cond * SIGMA32, dim=1, keepdim=True)

# ───────────────────────────────────────────────────────────────
# 2. PINN NETWORK DEFINITION
# ----------------------------------------------------------------
class PINN(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.linears = nn.ModuleList()
        for i in range(len(layers)-1):
            self.linears.append(nn.Linear(layers[i], layers[i+1]))
        self.activation = nn.Tanh()

    def forward(self, x):
        for lin in self.linears[:-1]:
            x = self.activation(lin(x))
        return self.linears[-1](x)

# Instantiate model
dim_hidden = 64
layers = [2] + [dim_hidden]*4 + [5]  # 5 hidden layers, output=5 (ux,uy,σxx,σyy,σxy)
model = PINN(layers).to(device)

# ───────────────────────────────────────────────────────────────
# 3. RESIDUAL FUNCTIONS
# ----------------------------------------------------------------
def pde_residual(xy: torch.Tensor, out: torch.Tensor):
    """
    Compute physics residuals:
      1) momentum x: ∂σxx/∂x + ∂σxy/∂y = 0
      2) momentum y: ∂σxy/∂x + ∂σyy/∂y = 0
      3) Hooke law consistency for σxx,σyy,σxy
    xy must have requires_grad=True
    out = model(xy)
    """
    ux = out[:,0:1]; uy = out[:,1:2]
    Sxx = out[:,2:3]; Syy = out[:,3:4]; Sxy = out[:,4:5]
    # Strain via autograd
    grads_ux = torch.autograd.grad(ux, xy, grad_outputs=torch.ones_like(ux), create_graph=True)[0]
    Exx = grads_ux[:,0:1]
    dux_dy = grads_ux[:,1:2]
    grads_uy = torch.autograd.grad(uy, xy, grad_outputs=torch.ones_like(uy), create_graph=True)[0]
    Eyy = grads_uy[:,1:2]
    duy_dx = grads_uy[:,0:1]
    Exy = 0.5*(dux_dy + duy_dx)
    # Hooke plane-strain
    Sxx_th = (2*mu + lmbd)*Exx + lmbd*Eyy
    Syy_th = (2*mu + lmbd)*Eyy + lmbd*Exx
    Sxy_th = 2*mu*Exy
    # Equilibrium residuals
    dSxx_dx = torch.autograd.grad(Sxx, xy, grad_outputs=torch.ones_like(Sxx), create_graph=True)[0][:,0:1]
    dSxy_dy = torch.autograd.grad(Sxy, xy, grad_outputs=torch.ones_like(Sxy), create_graph=True)[0][:,1:2]
    dSxy_dx = torch.autograd.grad(Sxy, xy, grad_outputs=torch.ones_like(Sxy), create_graph=True)[0][:,0:1]
    dSyy_dy = torch.autograd.grad(Syy, xy, grad_outputs=torch.ones_like(Syy), create_graph=True)[0][:,1:2]
    mom_x = dSxx_dx + dSxy_dy
    mom_y = dSxy_dx + dSyy_dy
    return [mom_x, mom_y, Sxx_th - Sxx, Syy_th - Syy, Sxy_th - Sxy]


def bc_residual(xy: torch.Tensor, out: torch.Tensor):
    """
    Compute traction residuals on boundary:
      tx - σ_n nx = 0, ty - σ_n ny = 0
    """
    nx = xy[:,0:1] / R_disk
    ny = xy[:,1:2] / R_disk
    Sxx = out[:,2:3]; Syy = out[:,3:4]; Sxy = out[:,4:5]
    tx = Sxx*nx + Sxy*ny
    ty = Sxy*nx + Syy*ny
    sigma_n = sigma_of_angle(xy)
    return tx - sigma_n*nx, ty - sigma_n*ny

# ───────────────────────────────────────────────────────────────
# 4. SAMPLING FUNCTIONS
# ----------------------------------------------------------------
def sample_domain(n):
    # Uniform sampling in disk via rejection or polar transform
    r = R_disk * torch.sqrt(torch.rand(n,1, device=device))
    theta = 2*np.pi * torch.rand(n,1, device=device)
    x = r * torch.cos(theta); y = r * torch.sin(theta)
    pts = torch.cat([x,y], dim=1)
    pts.requires_grad_(True)
    return pts

def sample_boundary(n):
    theta = 2*np.pi * torch.rand(n,1, device=device)
    x = R_disk * torch.cos(theta); y = R_disk * torch.sin(theta)
    pts = torch.cat([x,y], dim=1)
    pts.requires_grad_(True)
    return pts

# ───────────────────────────────────────────────────────────────
# 5. TRAINING LOOP
# ----------------------------------------------------------------
# Hyperparameters
num_domain   = 10000
num_boundary = 2000
lr           = 7.5e-4
epochs       = 30000
w_bc=5


optimizer = optim.Adam(model.parameters(), lr=lr)
pde_history, bc_history,loss_history = [], [],[]
for epoch in range(1, epochs+1):
    optimizer.zero_grad()
    # Sample points
    xy_dom = sample_domain(num_domain)
    xy_bnd = sample_boundary(num_boundary)
    # Forward pass
    out_dom = model(xy_dom)
    res_pde = pde_residual(xy_dom, out_dom)
    loss_pde = sum((r**2).mean() for r in res_pde)
    out_bnd = model(xy_bnd)
    tx_res, ty_res = bc_residual(xy_bnd, out_bnd)
    loss_bc  = tx_res.pow(2).mean() + ty_res.pow(2).mean()



    # Total loss (BC weighted ×10)
    loss = loss_pde + w_bc * loss_bc
    loss.backward()
    optimizer.step()
    # Logging
    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, PDE Loss: {loss_pde.item():.3e}, BC Loss: {loss_bc.item():.3e}, Total Loss: {loss.item():.3e}")
        pde_history.append(loss_pde.item())
        bc_history.append(loss_bc.item())
        loss_history.append(loss.item())

# ───────────────────────────────────────────────────────────────
# 6. POST-PROCESSING & PLOTTING
# ----------------------------------------------------------------
# Example: stress trace contour map
model.eval()
Ngrid = 250
x_vals = np.linspace(-R_disk, R_disk, Ngrid)
y_vals = np.linspace(-R_disk, R_disk, Ngrid)
X, Y = np.meshgrid(x_vals, y_vals)
pts = torch.tensor(np.column_stack([X.ravel(), Y.ravel()]), device=device, dtype=torch.float32)
with torch.no_grad():
    out = model(pts)
sx = out[:,2].cpu().numpy().reshape(X.shape)
sy = out[:,3].cpu().numpy().reshape(X.shape)
trace = sx + sy
mask = np.hypot(X, Y) > R_disk
trace[mask] = np.nan
plt.figure(); plt.contourf(X, Y, trace, 50, cmap='turbo'); plt.colorbar(); plt.title('Stress Trace'); plt.gca().set_aspect('equal'); plt.show()

plt.figure(figsize=(8,4))
plt.plot(np.arange(1, epochs/1000+1), pde_history, label='PDE Loss')
plt.plot(np.arange(1, epochs/1000+1), bc_history, label='BC Loss')
plt.plot(np.arange(1, epochs/1000+1), loss_history, label='Total Loss')
plt.xlabel('Epoch(*1000)')
plt.ylabel('Loss')
plt.yscale('log')
plt.title('Training Loss over Epochs')
plt.legend()
plt.grid(True)
plt.show()

# ───────────────────────────────────────────────────────────────
# 6-1. Displacement contour plots (ux, uy)
# ----------------------------------------------------------------
model.eval()

# 기존에 만든 그리드(X, Y)와 동일하게 사용
# pts: (Ngrid*Ngrid, 2), 안쪽/바깥쪽 마스크는 동일
pts = torch.tensor(
    np.column_stack([X.ravel(), Y.ravel()]),
    device=device, dtype=torch.float32
)

with torch.no_grad():
    out_disp = model(pts)  # 출력: [ux, uy, sxx, syy, sxy]

ux = out_disp[:, 0].detach().cpu().numpy().reshape(X.shape)
uy = out_disp[:, 1].detach().cpu().numpy().reshape(X.shape)

# 원판 바깥쪽 마스킹
mask = np.hypot(X, Y) > R_disk
ux = ux.astype(np.float64)  # NaN 대입을 위해 float64 권장
uy = uy.astype(np.float64)
ux[mask] = np.nan
uy[mask] = np.nan

# ux contour
plt.figure(figsize=(6, 5))
plt.contourf(X, Y, ux, 100, cmap='turbo')
plt.colorbar(label='u_x (mm)')  # 단위 표기는 상황에 맞게 조정
plt.title('Displacement $u_x$ Contour')
plt.gca().set_aspect('equal')
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.tight_layout()
plt.show()

# uy contour
plt.figure(figsize=(6, 5))
plt.contourf(X, Y, uy, 100, cmap='turbo')
plt.colorbar(label='u_y (mm)')
plt.title('Displacement $u_y$ Contour')
plt.gca().set_aspect('equal')
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.tight_layout()
plt.show()

# ───────────────────────────────────────────────────────────────
# 7. CIRCUMFERENTIAL VOLUMETRIC STRAIN (ε_v) AT r = r_sample
# ----------------------------------------------------------------
r_sample = 0.065  # [m] (must be <= R_disk)
theta = np.linspace(0, 2*np.pi, 360, endpoint=False).astype(np.float32)

# (x, y) points on the circle of radius r_sample
circle = np.column_stack([r_sample*np.cos(theta),
                          r_sample*np.sin(theta)]).astype(np.float32)

# Model inference (no gradients)
model.eval()
with torch.no_grad():
    circle_t = torch.from_numpy(circle).to(device)
    out_circ = model(circle_t)
    # outputs: [ux, uy, Sxx, Syy, Sxy]
    ux_c  = out_circ[:, 0].detach().cpu().numpy()
    uy_c  = out_circ[:, 1].detach().cpu().numpy()
    sx_c  = out_circ[:, 2].detach().cpu().numpy()
    sy_c  = out_circ[:, 3].detach().cpu().numpy()
    sxy_c = out_circ[:, 4].detach().cpu().numpy()

# Stress trace and volumetric strain (percent)
trace_c = sx_c + sy_c  # σxx + σyy  [MPa]
# NOTE: user intent kept: ε_v ≈ trace / (2*(λ+μ))
# Units: if E, λ, μ were in MPa and σ in MPa, 1e3 factor is for GPa→MPa conversion.
ev_circ = trace_c / (2.0 * (lmbd + mu)) * 100.0 / 1e3  # [%]

plt.figure(figsize=(6, 4))
plt.plot(np.degrees(theta), ev_circ, 'o-', ms=3)
plt.xlabel("Angle [deg]")
plt.ylabel("Volumetric strain ε_v [%]")
plt.title(f"ε_v along r = {r_sample:.3f} m")
plt.grid(True)
plt.tight_layout()
plt.show()

# ───────────────────────────────────────────────────────────────
# 6-2. Displacement magnitude |u| = sqrt(ux^2 + uy^2)
# ----------------------------------------------------------------
# ux, uy는 위에서 이미 계산됨 (X, Y와 같은 shape, 원판 바깥은 NaN 처리됨)

# 단위 스케일(기본: m). mm로 보려면 1e3로 바꾸고, unit="mm"로 설정.

scale = 1; unit = "mm"   # ← 주석 해제 시 mm로 표시

u_mag = np.hypot(ux+0.08750, uy-0.09784) * scale  # sqrt(ux^2 + uy^2)

plt.figure(figsize=(6, 5))
plt.contourf(X, Y, abs(u_mag), 100, cmap='turbo')
plt.colorbar(label=f'|u| ({unit})')
plt.title('Displacement Magnitude $|\\mathbf{u}|$')
plt.gca().set_aspect('equal')
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.tight_layout()
plt.show()